# Exercise 2 - Parallel Data Processing with Task Dependencies

**GOAL:** The goal of this exercise is to show how to pass object IDs into remote functions to encode dependencies between tasks.

In this exercise, we construct a sequence of tasks each of which depends on the previous mimicking a data parallel application. Within each sequence, tasks are executed serially, but multiple sequences can be executed in parallel.

In this exercise, you will use Ray to parallelize the computation below and speed it up.

### Concept for this Exercise - Task Dependencies

Suppose we have two remote functions defined as follows.

```python
@ray.remote
def f(x):
    return x
```

Arguments can be passed into remote functions as usual.

```python
>>> x1_id = f.remote(1)
>>> ray.get(x1_id)
1

>>> x2_id = f.remote([1, 2, 3])
>>> ray.get(x2_id)
[1, 2, 3]
```

**Object IDs** can also be passed into remote functions. When the function actually gets executed, **the argument will be a retrieved as a regular Python object**.

```python
>>> y1_id = f.remote(x1_id)
>>> ray.get(y1_id)
1

>>> y2_id = f.remote(x2_id)
>>> ray.get(y2_id)
[1, 2, 3]
```

So when implementing a remote function, the function should expect a regular Python object regardless of whether the caller passes in a regular Python object or an object ID.

**Task dependencies affect scheduling.** In the example above, the task that creates `y1_id` depends on the task that creates `x1_id`. This has the following implications.

- The second task will not be executed until the first task has finished executing.
- If the two tasks are scheduled on different machines, the output of the first task (the value corresponding to `x1_id`) will be copied over the network to the machine where the second task is scheduled.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import time

In [ ]:
ray.init(num_cpus=4, include_webui=False, ignore_reinit_error=True)

These are some helper functions that mimic an example pattern of a data parallel application.

**EXERCISE:** You will need to turn all of these functions into remote functions. When you turn these functions into remote function, you do not have to worry about whether the caller passes in an object ID or a regular object. In both cases, the arguments will be regular objects when the function executes. This means that even if you pass in an object ID, you **do not need to call `ray.get`** inside of these remote functions.

In [ ]:
def load_data(filename):
    time.sleep(0.1)
    return np.ones((1000, 100))

def normalize_data(data):
    time.sleep(0.1)
    return data - np.mean(data, axis=0)

def extract_features(normalized_data):
    time.sleep(0.1)
    return np.hstack([normalized_data, normalized_data ** 2])

def compute_loss(features):
    num_data, dim = features.shape
    time.sleep(0.1)
    return np.sum((np.dot(features, np.ones(dim)) - np.ones(num_data)) ** 2)

assert hasattr(load_data, 'remote'), 'load_data must be a remote function'
assert hasattr(normalize_data, 'remote'), 'normalize_data must be a remote function'
assert hasattr(extract_features, 'remote'), 'extract_features must be a remote function'
assert hasattr(compute_loss, 'remote'), 'compute_loss must be a remote function'

**EXERCISE:** The loop below takes too long. Parallelize the four passes through the loop by turning `load_data`, `normalize_data`, `extract_features`, and `compute_loss` into remote functions and then retrieving the losses with `ray.get`.

**NOTE:** You should only use **ONE** call to `ray.get`. For example, the object ID returned by `load_data` should be passed directly into `normalize_data` without needing to be retrieved by the driver.

In [ ]:
# Sleep a little to improve the accuracy of the timing measurements below.
time.sleep(2.0)
start_time = time.time()

losses = []
for filename in ['file1', 'file2', 'file3', 'file4']:
    inner_start = time.time()

    data = load_data(filename)
    normalized_data = normalize_data(data)
    features = extract_features(normalized_data)
    loss = compute_loss(features)
    losses.append(loss)
    
    inner_end = time.time()
    
    if inner_end - inner_start >= 0.1:
        raise Exception('You may be calling ray.get inside of the for loop! '
                        'Doing this will prevent parallelism from being exposed. '
                        'Make sure to only call ray.get once outside of the for loop.')

print('The losses are {}.'.format(losses) + '\n')
loss = sum(losses)

end_time = time.time()
duration = end_time - start_time

print('The loss is {}. This took {} seconds. Run the next cell to see '
      'if the exercise was done correctly.'.format(loss, duration))

**VERIFY:** Run some checks to verify that the changes you made to the code were correct. Some of the checks should fail when you initially run the cells. After completing the exercises, the checks should pass.

In [ ]:
assert loss == 4000
assert duration < 0.8, ('The loop took {} seconds. This is too slow.'
                        .format(duration))
assert duration > 0.4, ('The loop took {} seconds. This is too fast.'
                        .format(duration))

print('Success! The example took {} seconds.'.format(duration))

**EXERCISE:** Use the UI to view the task timeline and to verify that the four tasks were executed in parallel. To do this, run the following cell to generate a JSON file containing the profiling data. Open [chrome://tracing/](chrome://tracing/) in your browser, click on the "Load" button and load the JSON file.

- Using the **second** button, you can click and drag to **move** the timeline.
- Using the **third** button, you can click and drag to **zoom**. You can also zoom by holding \"alt\" and scrolling.

**NOTE:** If you are on Binder, use the link below to download the file on you laptop (right click and select "Save As") and then load it in [chrome://tracing/](chrome://tracing/).

**NOTE:** The timeline visualization will only work in **Chrome**.

In [ ]:
ray.global_state.chrome_tracing_dump(filename="/tmp/timeline02.json")

Download link for Binder: http://localhost:8000/timeline02.json

### Application: Parallel web-scraping

One useful application of what we have learned so far is to scrape information from the web. We will illustrate this in a toy setting, but the same principles apply on a large scale where crawling through websites, parsing them and extracting useful content (e.g. for building a search index or populating a database) is often very computationally demanding.

We break up the process into multiple steps. We first grab the raw HTML of the website using Python's requests package. Then, we use BeautifulSoup to parse the HTML to find the relevant information. Finally, we populate a pandas DataFrames so that we are able to work with the data.

To demonstrate this, we scrape google search results of ray and other related libraries to find other resources to help us learn ray and the other related libraries.

In [ ]:
from bs4 import BeautifulSoup
import requests

import pandas as pd

The following function uses these libraries to submit a Google search query and parse the results.

In [ ]:
def query_google(keyword):
    url = "https://google.com/search?q={}".format(keyword)
    response = requests.get(url)
    return response

def parse_google_response(keyword, response):
    soup = BeautifulSoup(response.text, 'lxml')
    df = pd.DataFrame(columns=['title', 'link', 'description'])
    for g in soup.find_all(class_='g'):
        entry = {}
        headers = g.find_all(class_='r')
        # Sometimes results have no headers
        if len(headers) != 1:
            continue
        entry['title'] = headers[0].text
        entry['link'] = headers[0].find('a').get('href')[7:]
        description = g.find_all(class_='st')
        entry['description'] = description[0].text if len(description) > 0 else ""
        df = df.append(pd.DataFrame(entry, index=[0]))
    
    df['library'] = keyword
    return df

def get_results(keyword):
    response = query_google("learn {}".format(keyword))
    return parse_google_response(keyword, response)

Let's try this out to get results for some ray related topics serially.

In [ ]:
start = time.time()
keywords = ["ray", "rllib", "tune", "modin", "plasma", "arrow"]
results = []
for keyword in keywords:
    df = get_results(keyword)
    results.append(df)
    
df = pd.concat(results)
duration = time.time() - start
print("Constructing the dataframe took {} seconds.".format(duration))

**EXERCISE**: Speed up the above serial query by making `get_results` a remote function in order to scrape google results in parallel. Then, see a sample of the data scraped below and feel free to play with the data to find other resources to learn more about these libraries!

In [ ]:
df